In [ ]:
# functionality/common_functions.py
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import regex as re
 
# Utility functions with wait statements
def expand_filter(driver, wait, choice):
    time.sleep(5)  # Ensure the page is loaded before interacting
    xpath = f"//h3[contains(text(), '{choice}')]/following-sibling::button[@aria-label='Expand content']"
    wait.until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
    time.sleep(2)
 
def select_checkbox(driver, wait, label):
    xpath = f"//span[text()='{label}']/preceding-sibling::input[@type='checkbox']"
    checkbox = wait.until(EC.element_to_be_clickable((By.XPATH, xpath)))
    driver.execute_script("arguments[0].click();", checkbox)
    time.sleep(2)
 
# Create a dictionary to map month names to numbers
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
 
def date_range_selector(driver, wait, label, day, month, year):
    button = wait.until(EC.element_to_be_clickable((By.XPATH, f"//span[.='{label}']/ancestor::button")))
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)
 
    # Wait for calendar to be visible
    calendar = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@role='dialog']")))
    current_month_year = driver.find_element(By.XPATH, "//div[@role='dialog']//div[@class='rdp-month_caption']").text.split(' ')
    current_month, current_year = current_month_year[0], current_month_year[1]
 
    # Get numeric values for current and target months
    current_month_num = month_map[current_month]
    target_month_num = month_map[month]
 
    # Swipe to the correct month for the target date
    while not (current_month_num == target_month_num and current_year == str(year)):
        if current_month_num < target_month_num or (current_month_num == target_month_num and int(current_year) < year):
            # Move to the next month
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@role='dialog']//nav/button[@aria-label='Go to the Next Month']")))
            next_button.click()
        else:
            # Move to the previous month
            prev_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[@role='dialog']//nav/button[@aria-label='Go to the Previous Month']")))
            prev_button.click()
 
        time.sleep(1)
        current_month_year = driver.find_element(By.XPATH, "//div[@role='dialog']//div[@class='rdp-month_caption']").text.split(' ')
        current_month, current_year = current_month_year[0], current_month_year[1]
        current_month_num = month_map[current_month]  # Get numeric value of the current month
 
    target_date = f"{year}-{str(target_month_num).zfill(2)}-{str(day).zfill(2)}"    
    date_button_xpath = f"//div[@role='dialog']//table//tbody//tr//td[@data-day='{target_date}']//button"
    date_button = wait.until(EC.element_to_be_clickable((By.XPATH, date_button_xpath)))
 
    # Ensure the button is scrolled into view and then click it
    driver.execute_script("arguments[0].scrollIntoView();", date_button)  # Scroll the element into view
    time.sleep(1)  # Wait a moment to ensure it is in view
    date_button.click()  # Click the date button
 
    time.sleep(2)
    span_xpath = "//span[contains(., 'Powered by sales intelligence')]"
    element = wait.until(EC.element_to_be_clickable((By.XPATH, span_xpath)))
    element.click()
 
def num_of_rows_displayed(driver, wait):
    number_button = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//button[contains(@class,'text-white')]"))
    )
    button_text = number_button.text
    return int(re.search(r'\d+', button_text).group())
 
def click_dropdown(driver,wait,label,pos):
    xpath = f"//h3[contains(text(), '{label}')]/ancestor::div[1]/following-sibling::div//ul//li{[pos]}//div//span//span"
    wait.until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
    time.sleep(2)
 
def set_column_toggles(driver, wait,column_toggles,check_type):
    wait.until(EC.element_to_be_clickable((By.XPATH, "//thead//tr//button"))).click()
 
    # Loop through each column toggle
    for column_name, desired_state in column_toggles.items():
        # Dynamically build the XPath for the toggle button
        if((column_name == 'Account Name' and check_type=='Accounts') or (column_name == 'Contact Name' and check_type=='Contacts') ):
            continue
        toggle_xpath = f"//div[span[contains(text(), '{column_name}')]]/div//input"
        toggle_element = wait.until(EC.element_to_be_clickable((By.XPATH, toggle_xpath)))
       
        # Check the current state of the toggle
        current_state = toggle_element.get_attribute("aria-checked").lower() == "true"
       
        if current_state != desired_state:
            driver.execute_script("arguments[0].click();", toggle_element)
            print(f"{column_name} set to {'✅ Enabled' if desired_state else '❌ Disabled'}")
        else:
            print(f"{column_name} already {'✅ Enabled' if desired_state else '❌ Disabled'}")
   
    # Wait for the close button and click it
    close_button = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//div[@class='h-16 p-6 w-full flex justify-between items-center transition-opacity duration-200 shadow-md opacity-100 delay-200']//button"))
    )
    close_button.click()
    driver.refresh()
 
def find_column_no(driver, wait, column_name):
    all_header_column_elements = wait.until(
        EC.presence_of_all_elements_located((By.XPATH, "//thead//th//span"))
    )
 
    # Get the number of columns
    no_of_columns = len(all_header_column_elements)
    print(f"no of columns: {no_of_columns}")
 
    column_index = None
    for i in range(no_of_columns):
        column_text = all_header_column_elements[i].text
        if column_text == f"{column_name}":
            column_index = i+1  
            return column_index
       
 
def iterate_over_pages(driver, wait, no_of_pages, column_index):
    for page_num in range(1, no_of_pages + 1):
        new_url = re.sub(r'(?<=page=)\d+', str(page_num), driver.current_url)
        driver.get(new_url)
        time.sleep(2)  # Wait for the page to load
        # Wait for the page to load (check for a specific element on the page to confirm)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//tbody//tr"))
        )
        time.sleep(2)
 
        fetched_info = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, f"//tbody//tr//td[{column_index}]//span//div/span//span[2]"))
        )
 